# Physical Pages Management

### Lista de paginas

```c
struct PageInfo *pages;
```

### PageInfo

Esta estructura es la que modeliza una pagina. Es el siguiente <b>struct</b>:

Contiene dos atributos:
- <b>pp_link</b>: Es el link hacia la siguiente pagina libre.
- <b>pp_ref</b>: Es la cantidad de referencias hacia esta pagina.

```c
struct PageInfo {
	// Next page on the free list.
	struct PageInfo *pp_link;

	// pp_ref is the count of pointers (usually in page table entries)
	// to this page, for pages allocated using page_alloc.
	// Pages allocated at boot time using pmap.c's
	// boot_alloc do not have valid reference count fields.

	uint16_t pp_ref;
};

```

### Lista de paginas libres

Observando el siguiente esquema vemos que usando el atributo <b>pp_link</b> cada elemento del array <b>pages</b>, osea cada pagina apunta a la siguiente libre. Ademas se muestra como la variable <b>page_free_list</b> apunta a la primera pagina libre.

<img src="Imagenes/pages.jpg">

### Obtener una pagina libre

Esto se logra haciendo uso de la funcion <b>page_alloc</b>. Esta funcion obtiene una pagina libre de la lista de paginas para que la pueda utilizar un programa.

```c
struct PageInfo *
page_alloc(int alloc_flags)
{	
	// freepage := la primera pagina en la lista de paginas libres
	struct PageInfo* freepage = page_free_list;

	// No hay una pagina fisica libre, por lo tanto retornamos NULL
	if (freepage == NULL) return NULL;

	// actualizamos la lista sacando la pagina que vamos a retornar
	page_free_list = page_free_list->pp_link;

	// si 'alloc_flags' indica ALLOC_ZERO, entonces tenemos que
	// rellenar con ceros
	if (alloc_flags & ALLOC_ZERO){
		// agarramos la direccion virtual de la pagina
		void* va = page2kva(freepage);
		// para luego inicialziarla completamente con ceros
		memset(va, 0, PGSIZE);
	} 
	
	// incializar el contador de las referencias de esta pagina
	freepage->pp_ref = 0;
	
	// limpiamos el valor de pp_link, ya que no lo necesita mas 
	// debido a que ya no pertenece a la lista
	freepage->pp_link = NULL;

	// retornamos finalmente la info de la pagina alocada
	return freepage;
}
```

### Inicializar la lista de paginas

Esta inicializacion es simplemente indicarle pagina cual es su siguiente libre. Y inicializar la cantidad de referencias a cada una de ellas en 0.

```c
void
page_init(void)
{
	int top_boot_alloc = (int)PADDR(boot_alloc(0));
	for (size_t i = 0 ; i < npages; i++) {
	/************ Ilustracion de la memoria fisica, para identificar los sectores libres *******************
	  | 0 -------- npages_basemem -------- IOPHYSMEM --------- EXTPHYSMEM -------- top_boot_alloc -------- |
	*******************************************************************************************************/
		// validamos de no colocar una pagina que no esta libre a la lista de paginas libres
		if (
			((i > 0 && i<npages_basemem) ||
			(i >= top_boot_alloc/PGSIZE))
		) {
			// inicializamos el contador de referencias de la pagina en cero.
			pages[i].pp_ref = 0;
			// introducimos esta pagina a el principio de la lista de paginas.
			pages[i].pp_link = page_free_list;
			page_free_list = &pages[i];
		}
	}
}
```

### Liberar una pagina

Esto se logra haciendo uso de la funcion <b>page_free</b>. Esta funcion agarra una pagina que no pertenece a la lista de paginas libre, y la vuelve a poner en ella. Para que posteriormente pueda ser utilizada por otro programa.

```c
void
page_free(struct PageInfo *pp)
{
	// Si el contador de referencias de esta pagina es mayor a cero
	// no la podemos liberar ya que los que estan apuntando hacia ella
	// quedarian apuntando a algo invalido, por lo tanto evitamos esto
	// emitiendo un 'panic' con el mensaje correspondiente
	if ((pp->pp_ref != 0) || (pp->pp_link != NULL))
		panic ("Tried to free memory without pp_ref!=0 or pp_link!=NULL");

	// introducimos la pagina a la lista de paginas libres, para que 
	// pueda ser usada posteriormente.
	pp->pp_link = page_free_list;
	page_free_list = pp;
}
```

# Memoria Virtual

Los programas y el mismo kernel utilizan direcciones virtuales, no fisicas. Estas direcciones virtuales son traducidas a direcciones fisicas como luego veremos. Este sistema de memoria hace que el programa crea que tiene infinita memoria.

Esquema de como funciona el sistema de traduccion.

<img src="Imagenes/x86-addr-translation.gif">

Integrantes de este modelo:
- <b>Page Directory</b>: Este es una tabla que contiene las direcciones de todas las tablas de paginas.
- <b>Page Table</b>: Esta es una tabla que contiene las direcciones fisicas (phisycal address) de las paginas (ppa).  


### Virtual Address

<img src="Imagenes/x86-addr-format.gif">

En la foto vemos la estructura que tiene una direccion virtual. 
- <b>PDE</b> (10 bits): Es el indice en el directorio de paginas.
- <b>PTE</b> (10 bits): Es el indice en la tabla de paginas.
- <b>Offset</b> (12 bits): Es el offset en la pagina.